In [1]:
# import package
from datetime import date,timedelta
from urllib.request import urlopen
from dateutil import rrule
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np
import json
import time

%matplotlib inline

In [2]:
# 爬取每月股價的目標網站並包裝成函式
def craw_one_month(stock_number,date):
    url = (
        "http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+
        date.strftime('%Y%m%d')+
        "&stockNo="+
        str(stock_number)
    )
    data = json.loads(urlopen(url).read())
    return pd.DataFrame(data['data'],columns=data['fields'])

In [9]:
# 根據使用者輸入的日期，以月為單位，重複呼叫爬取月股價的函式
def craw_stock(stock_number, start_month):
    b_month = date(*[int(x) for x in start_month.split('-')])
    now = datetime.datetime.now().strftime("%Y-%m-%d")         # 取得現在時間
    e_month = date(*[int(x) for x in now.split('-')])
    
    result = pd.DataFrame()
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=b_month, until=e_month):
        result = pd.concat([result,craw_one_month(stock_number,dt)],ignore_index=True)
        time.sleep(5000.0/1000.0);
    
    return result

df = craw_stock(2609,"2020-01-01")
df.set_index("日期", inplace=True)
df

,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
日期,,,,,,,,
109/01/02,"1,124,232","8,183,676",7.26,7.29,7.25,7.28,+0.07,507
109/01/03,"1,665,314","12,156,458",7.36,7.36,7.26,7.32,+0.04,721
109/01/06,"1,375,475","10,064,850",7.34,7.36,7.26,7.26,-0.06,459
109/01/07,"959,325","6,988,439",7.28,7.30,7.26,7.28,+0.02,404
109/01/08,"768,919","5,558,814",7.28,7.28,7.20,7.20,-0.08,362
...,...,...,...,...,...,...,...,...
110/08/16,"302,376,585","39,853,392,553",135.00,136.50,125.00,126.00,-7.00,"122,425"
110/08/17,"187,972,500","24,001,265,341",129.00,131.00,123.00,124.50,-1.50,"81,289"
110/08/18,"325,683,248","42,428,534,457",123.50,136.50,121.00,136.50,+12.00,"142,925"


In [10]:
df.to_csv('2609_20200101.csv', encoding='utf_8_sig')